# Análisis Exploratorio de Datos (EDA)
## Logística, Inventario y Transacciones

En este notebook exploraremos tres conjuntos de datos relacionados con operaciones de logística y gestión de inventario.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Cargar los datos
transacciones = pd.read_csv('/Users/juangomez/Downloads/transacciones_logistica_v2.csv')
inventario = pd.read_csv('/Users/juangomez/Downloads/inventario_central_v2.csv')
feedback = pd.read_csv('/Users/juangomez/Downloads/feedback_clientes_v2.csv')

print("="*80)
print("DATOS CARGADOS EXITOSAMENTE")
print("="*80)
print(f"\nTransacciones: {transacciones.shape}")
print(f"Inventario: {inventario.shape}")
print(f"Feedback: {feedback.shape}")

DATOS CARGADOS EXITOSAMENTE

Transacciones: (10000, 10)
Inventario: (2500, 8)
Feedback: (4500, 9)


In [2]:
# Funciones
def print_table(title: str, dataframe: pd.DataFrame, rows: int = 10, show_index: bool = False):
    """
    Muestra un DataFrame en forma de tabla simple.
    Parámetros:
      - title: Título que se mostrará arriba de la tabla.
      - dataframe: pd.DataFrame a mostrar.
      - rows: Número de filas a mostrar (por defecto 10).
      - show_index: Mostrar índice (por defecto False).
    """
    print(f"\n{title}")
    print(f"{dataframe.shape[0]} filas x {dataframe.shape[1]} columnas\n")

    if dataframe.empty:
        print("DataFrame vacío")
        return

    df_show = dataframe.head(rows)
    if not show_index:
        df_show = df_show.reset_index(drop=True)

    display(df_show)

## 1. RESUMEN GENERAL DE DATOS

In [ ]:
transacciones['Fecha_Venta'] = pd.to_datetime(transacciones['Fecha_Venta'], format='%d/%m/%Y')



print("\n" + "="*80)
print("CONVERSION DE FECHAS COMPLETADA")
print("="*80)
# Verificar conversiones
print("\nTransacciones - Tipo de dato de 'Fecha_Venta':", transacciones['Fecha_Venta'].dtype)
print("Inventario - Tipo de dato de 'Ultima_Revision':", inventario['Ultima_Revision'].dtype)
print("="*80)

## 2. Análisis de Inventario

In [3]:
print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - INVENTARIO")
print("="*80)
print(f"Inventario: {inventario.shape}")
print(inventario.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO")
print("="*80)
print(inventario.select_dtypes(include=['number']).describe())


INFORMACIÓN DETALLADA - INVENTARIO
Inventario: (2500, 8)
<class 'pandas.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SKU_ID              2500 non-null   str    
 1   Categoria           2500 non-null   str    
 2   Stock_Actual        2400 non-null   float64
 3   Costo_Unitario_USD  2500 non-null   float64
 4   Punto_Reorden       2500 non-null   int64  
 5   Lead_Time_Dias      2097 non-null   str    
 6   Bodega_Origen       2500 non-null   str    
 7   Ultima_Revision     2500 non-null   str    
dtypes: float64(2), int64(1), str(5)
memory usage: 156.4 KB
None

ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO
       Stock_Actual  Costo_Unitario_USD  Punto_Reorden
count   2400.000000         2500.000000    2500.000000
mean     995.487083         1105.788816     198.046400
std      597.689734        16989.836953      57.182355
min      -50.000000   

### 2.2 Limpieza de inventarios

In [5]:
# Función de saneamiento de la información.
def sanitize_inventario(dataframe: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Sanea el DataFrame de inventario y devuelve una dupla:
    (DataFrame saneado, DataFrame resumen de filas afectadas por cada proceso).
    """
    dataframe = dataframe.copy()
    report = {}

    # 1. Remover espacios en blanco en columnas categóricas
    cat_cols = dataframe.select_dtypes(include=['object', 'string']).columns
    affected_spaces = 0
    for col in cat_cols:
        mask = dataframe[col].notnull() & (dataframe[col] != dataframe[col].astype(str).str.strip())
        affected_spaces += mask.sum()
        dataframe[col] = dataframe[col].where(dataframe[col].isnull(),
                                              dataframe[col].astype(str).str.strip())
    report["Remover espacios en blanco"] = int(affected_spaces)

    # 2. Reemplazar cadenas vacías por NA
    before_empty = dataframe[cat_cols].isin(['']).sum().sum()
    dataframe[cat_cols] = dataframe[cat_cols].replace({'': pd.NA})
    report["Reemplazar cadenas vacías por NA"] = int(before_empty)

    # 3. Conversión de fechas en 'Ultima_Revision'
    before_invalid_dates = dataframe['Ultima_Revision'].isnull().sum()
    dataframe['Ultima_Revision'] = pd.to_datetime(dataframe['Ultima_Revision'], format='%Y-%m-%d', errors='coerce').dt.strftime('%d/%m/%Y')
    dataframe['Ultima_Revision'] = pd.to_datetime(dataframe['Ultima_Revision'], dayfirst=True, errors='coerce')
    after_invalid_dates = dataframe['Ultima_Revision'].isnull().sum()
    report["Conversión de fechas inválidas a NaT"] = int(after_invalid_dates - before_invalid_dates)

    # 4. Stock negativo a positivo
    neg_stock = (dataframe['Stock_Actual'] < 0).sum()
    dataframe.loc[dataframe['Stock_Actual'] < 0, 'Stock_Actual'] = dataframe.loc[dataframe['Stock_Actual'] < 0, 'Stock_Actual'].abs()
    report["Stock negativo convertido a positivo"] = int(neg_stock)

    # 5. Normalización de Bodega_Origen
    mapping_bodegas = {
        'norte': 'norte', 'Norte': 'norte', 'Sur': 'sur', 'sur': 'sur',
        'ZONA_FRANCA': 'zona franca', 'Occidente': 'occidente', 'occidente': 'occidente', 'BOD-EXT-99': 'bod-ext-99'
    }
    before_bodega = (dataframe['Bodega_Origen'].notnull() & dataframe['Bodega_Origen'].isin(mapping_bodegas.keys())).sum()
    dataframe['Bodega_Origen'] = dataframe['Bodega_Origen'].astype("string").str.strip().replace(mapping_bodegas)
    after_bodega = (dataframe['Bodega_Origen'].notnull() & dataframe['Bodega_Origen'].isin(mapping_bodegas.values())).sum()
    report["Normalización de Bodega_Origen"] = int(before_bodega)

    # 6. Normalización de Categoria
    mapping_categorias = {
        'Laptops': 'laptops', 'Monitores': 'monitores', 'Smartphones': 'smartphones', 'Tablets': 'tablets',
        'Accesorios': 'accesorios', 'smart-phone': 'smartphones', 'LAPTOP': 'laptops', '???': 'otros'
    }
    before_cat = (dataframe['Categoria'].notnull() & dataframe['Categoria'].isin(mapping_categorias.keys())).sum()
    dataframe['Categoria'] = dataframe['Categoria'].astype("string").str.strip().replace(mapping_categorias)
    after_cat = (dataframe['Categoria'].notnull() & dataframe['Categoria'].isin(mapping_categorias.values())).sum()
    report["Normalización de Categoria"] = int(before_cat)

    # 7. Rellenar nulos en Lead_Time_Dias y normalizar texto
    affected_lead = 0
    for col in ('Lead_Time_Dias', 'leads_time_dias'):
        if col in dataframe.columns:
            mask = dataframe[col].isnull() | (dataframe[col].astype(str).str.strip() == '')
            affected_lead += mask.sum()
            dataframe[col] = dataframe[col].astype("string").str.strip().replace({'': pd.NA})
            dataframe[col] = dataframe[col].fillna('sin_definir').astype("string").str.lower()
    report["Rellenar nulos y normalizar Lead_Time_Dias"] = int(affected_lead)

    # 8. Renombrar y rellenar nulos en Stock_Actual
    affected_stock_null = 0
    if 'stock_actual' in dataframe.columns and 'Stock_Actual' not in dataframe.columns:
        dataframe = dataframe.rename(columns={'stock_actual': 'Stock_Actual'})
    if 'Stock_Actual' in dataframe.columns:
        mask = dataframe['Stock_Actual'].isnull()
        affected_stock_null = mask.sum()
        dataframe['Stock_Actual'] = pd.to_numeric(dataframe['Stock_Actual'], errors='coerce').fillna(0)
    report["Rellenar nulos en Stock_Actual con 0"] = int(affected_stock_null)

    inventarios_report = pd.DataFrame(list(report.items()), columns=["Proceso", "Filas_afectadas"])
    return dataframe, inventarios_report

In [6]:
# Ejecutar saneamiento y obtener reporte (devuelve: DataFrame saneado, DataFrame reporte)
inventario, inventarios_report = sanitize_inventario(inventario)

print("\nResumen de procesos de saneamiento (Inventario):")
display(inventarios_report)

# Resumen adicional en formato tabla: absoluto y porcentaje respecto al total de filas del inventario
total_rows = inventario.shape[0]
summary = inventarios_report.copy()
summary['Porcentaje'] = (summary['Filas_afectadas'] / total_rows * 100).round(2)
summary = summary.sort_values('Filas_afectadas', ascending=False).reset_index(drop=True)

print("\nResumen (absoluto y % respecto al inventario):")
display(summary.style.format({'Porcentaje': '{:.2f}%'}))

print("\n" + "="*80)
print("SANITIZACIÓN DE DATOS COMPLETADA - INVENTARIO")
print("="*80)

print("\n" + "="*80)
print("INFORMACIÓN DETALLADA - INVENTARIO")
print("="*80)
print(f"Inventario: {inventario.shape}")
print(inventario.info())
print("\n" + "="*80)
print("ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO")
print("="*80)
print(inventario.select_dtypes(include=['number']).describe())
print_table("PRIMERAS FILAS - INVENTARIO", inventario.head())


Resumen de procesos de saneamiento (Inventario):


,Proceso,Filas_afectadas
0,Remover espacios en blanco,0
1,Reemplazar cadenas vacías por NA,0
2,Conversión de fechas inválidas a NaT,0
3,Stock negativo convertido a positivo,60
4,Normalización de Bodega_Origen,2500
5,Normalización de Categoria,2500
6,Rellenar nulos y normalizar Lead_Time_Dias,403
7,Rellenar nulos en Stock_Actual con 0,100



Resumen (absoluto y % respecto al inventario):


,Proceso,Filas_afectadas,Porcentaje
0,Normalización de Bodega_Origen,2500,100.00%
1,Normalización de Categoria,2500,100.00%
2,Rellenar nulos y normalizar Lead_Time_Dias,403,16.12%
3,Rellenar nulos en Stock_Actual con 0,100,4.00%
4,Stock negativo convertido a positivo,60,2.40%
5,Remover espacios en blanco,0,0.00%
6,Reemplazar cadenas vacías por NA,0,0.00%
7,Conversión de fechas inválidas a NaT,0,0.00%



SANITIZACIÓN DE DATOS COMPLETADA - INVENTARIO

INFORMACIÓN DETALLADA - INVENTARIO
Inventario: (2500, 8)
<class 'pandas.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SKU_ID              2500 non-null   str           
 1   Categoria           2500 non-null   string        
 2   Stock_Actual        2500 non-null   float64       
 3   Costo_Unitario_USD  2500 non-null   float64       
 4   Punto_Reorden       2500 non-null   int64         
 5   Lead_Time_Dias      2500 non-null   string        
 6   Bodega_Origen       2500 non-null   string        
 7   Ultima_Revision     2500 non-null   datetime64[us]
dtypes: datetime64[us](1), float64(2), int64(1), str(1), string(3)
memory usage: 156.4 KB
None

ESTADÍSTICAS DESCRIPTIVAS NUMÉRICAS - INVENTARIO
       Stock_Actual  Costo_Unitario_USD  Punto_Reorden
count   2500.000000         2500.000000 

,SKU_ID,Categoria,Stock_Actual,Costo_Unitario_USD,Punto_Reorden,Lead_Time_Dias,Bodega_Origen,Ultima_Revision
0,PROD-1000,smartphones,0.0,870.38,259,25-30 días,norte,2025-11-17
1,PROD-1001,accesorios,476.0,1397.26,169,25-30 días,norte,2024-03-05
2,PROD-1002,monitores,1209.0,611.62,214,5,sur,2024-06-21
3,PROD-1003,smartphones,1825.0,145.94,187,10,sur,2025-01-07
4,PROD-1004,smartphones,1713.0,77.78,105,5,sur,2024-07-04


## 3. Análisis de Transacciones

### 3.1 Análisis de valores nulos, duplicados y negativos